In [144]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import time
from sklearn.metrics import mean_absolute_error, r2_score
import numpy as np
from sklearn import preprocessing

In [149]:
df = pd.read_csv('leg234_data.csv')


<div>
<img src="images/impurity.png" width="400"/>
<img src="images/permutation.png" width="400"/>
</div>

In [6]:
df = df.drop(columns=['NMEA.Wave_Height', 'PCO2.H2O_mmm', 'PCO2.atm_cond', 'date', 'FerryBox.C3_CDOMFluorescence'])

In [145]:
df_sat = df[['LATITUDE', 'LONGITUD', 'PDMEAN', 'FerryBox.Optode_Saturation', 'time', 'NMEA.Wind_Angle', 'NMEA.Wind_Speed', 'TOTAL']]
df_time = df[['LATITUDE', 'LONGITUD', 'PDMEAN', 'time', 'TOTAL']]
df_wind = df[['LATITUDE', 'LONGITUD', 'PDMEAN', 'NMEA.Wind_Angle', 'TOTAL']]



names = df_sat.columns
# Create the Scaler object
scaler = preprocessing.StandardScaler()
# Fit your data on the scaler object
scaled_df = scaler.fit_transform(df_sat)
df_sat = pd.DataFrame(scaled_df, columns=names)

In [130]:
df = df[['LATITUDE', 'LONGITUD', 'PDMEAN', 'TOTAL']]
df

,LATITUDE,LONGITUD,PDMEAN,TOTAL
0,27.15121,-16.94351,7.5,0.0000
1,27.15121,-16.94351,15.0,6.6924
2,27.15121,-16.94351,25.0,27.0674
3,27.15121,-16.94351,35.0,21.1011
4,27.15121,-16.94351,45.0,22.7452
...,...,...,...,...
219694,23.19454,-40.66192,685.0,35.0402
219695,23.19454,-40.66192,695.0,45.1827
219696,23.19454,-40.66192,705.0,351.3631
219697,23.19454,-40.66192,715.0,42.6268


In [157]:
y = df.iloc[:, df.columns == 'TOTAL']
X = df.iloc[:, df.columns != 'TOTAL']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=123)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, 
    test_size=0.25, random_state=123) 

In [80]:
def smape(A, F):
    return 1/len(A) * np.sum(2 * np.abs(F - A) / (np.abs(A) + np.abs(F)))

In [158]:
forest = RandomForestRegressor(random_state=0)
forest.fit(X_train, y_train.values.ravel())

RandomForestRegressor(random_state=0)

In [156]:
forest_pred = forest.predict(X_val)
print('MAE score: ', mean_absolute_error(y_val, forest_pred))
print('R2 score: ', r2_score(y_val, forest_pred))
print('Smape score: ', smape(y_val, forest_pred.reshape(41194,1)))

MAE score:  16.74552681825023
R2 score:  0.7104052370422905
Smape score:  TOTAL    0.362625
dtype: float64


In [140]:
forest_pred = forest.predict(X_val)
print('MAE score: ', mean_absolute_error(y_val, forest_pred))
print('R2 score: ', r2_score(y_val, forest_pred))
print('Smape score: ', smape(y_val, forest_pred.reshape(41194,1)))

MAE score:  16.712684409598484
R2 score:  0.7252393902802352
Smape score:  TOTAL    0.366244
dtype: float64


In [78]:
forest_pred = forest.predict(X_val)
print('MAE score: ', mean_absolute_error(y_val, forest_pred))
print('R2 score: ', r2_score(y_val, forest_pred))
print('Smape score: ', smape(y_val, forest_pred.reshape(41194,1)))

MAE score:  16.728195075472158
R2 score:  0.7239675581020903
Smape score:  TOTAL    0.36199
dtype: float64


In [82]:
forest_pred = forest.predict(X_val)
print('MAE score: ', mean_absolute_error(y_val, forest_pred))
print('R2 score: ', r2_score(y_val, forest_pred))
print('Smape score: ', smape(y_val, forest_pred.reshape(41194,1)))

MAE score:  16.77536834907996
R2 score:  0.7235152499463483
Smape score:  TOTAL    0.366978
dtype: float64


In [83]:
print('Parameters currently in use:\n')
print(forest.get_params())

Parameters currently in use:

{'bootstrap': True, 'ccp_alpha': 0.0, 'criterion': 'mse', 'max_depth': None, 'max_features': 'auto', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': 0, 'verbose': 0, 'warm_start': False}


In [85]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)
{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}


{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}

In [87]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:880: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


RandomizedSearchCV(cv=3, estimator=RandomForestRegressor(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   random_state=42, verbose=2)

In [90]:
rf_random.best_params_

{'n_estimators': 1800,
 'min_samples_split': 5,
 'min_samples_leaf': 2,
 'max_features': 'sqrt',
 'max_depth': 60,
 'bootstrap': True}

In [93]:
best_random = rf_random.best_estimator_

In [99]:
r_pred = best_random.predict(X_val)
print('MAE score: ', mean_absolute_error(y_val, r_pred))
print('R2 score: ', r2_score(y_val, r_pred))
print('Smape score: ', smape(y_val, r_pred.reshape(41194,1)))

MAE score:  16.59704493643191
R2 score:  0.7460633937859095
Smape score:  TOTAL    0.422385
dtype: float64


In [126]:
clf = RandomForestRegressor(n_estimators=50, random_state=0, criterion='mae')

In [127]:
clf.fit(X_train, y_train.values.ravel())

KeyboardInterrupt: 

In [114]:
clf_pred = clf.predict(X_val)
print('MAE score: ', mean_absolute_error(y_val, clf_pred))
print('R2 score: ', r2_score(y_val, clf_pred))
print('Smape score: ', smape(y_val, clf_pred.reshape(41194,1)))

MAE score:  23.756654483336554
R2 score:  0.5835083517851651
Smape score:  TOTAL    0.569527
dtype: float64


In [ ]:
clf_pred = clf.predict(X_val)
print('MAE score: ', mean_absolute_error(y_val, clf_pred))
print('R2 score: ', r2_score(y_val, clf_pred))
print('Smape score: ', smape(y_val, clf_pred.reshape(41194,1)))